In [7]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D,Flatten,Dense

In [8]:
mnist = tf.keras.datasets.mnist
(X_train , Y_train) , (X_test , Y_test) = mnist.load_data()
X_train = X_train / 255.0
X_test = X_test / 255.0
X_train = X_train[..., tf.newaxis]
X_test = X_test[..., tf.newaxis]
print(X_train[0].shape)

(28, 28, 1)


In [9]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test)).batch(32).prefetch(tf.data.AUTOTUNE)

In [10]:
class MyMnistModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = Conv2D(32, 3, activation="relu")
        self.flatten = Flatten()
        self.d1 = Dense(128, activation="relu")
        self.d2 = Dense(32, activation="relu")
        self.d3 = Dense(10)

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        x = self.d2(x)
        x = self.d3(x)
        return x
model = MyMnistModel()

In [11]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

In [12]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [13]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

@tf.function
def test_step(images, labels):
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [15]:
EPOCHS = 6

for epoch in range(EPOCHS):
    train_loss.reset_state()
    train_accuracy.reset_state()
    test_loss.reset_state()
    test_accuracy.reset_state()

    for images, labels in train_dataset:
      train_step(images, labels)

    for test_images, test_labels in test_dataset:
      test_step(test_images, test_labels)

    print(
      f'Epoch {epoch + 1}, '
      f'Loss: {train_loss.result()}, '
      f'Accuracy: {train_accuracy.result()}, '
      f'Test Loss: {test_loss.result()}, '
      f'Test Accuracy: {test_accuracy.result()}'
    )

Epoch 1, Loss: 0.01, Accuracy: 0.9981499910354614, Test Loss: 0.08622753620147705, Test Accuracy: 0.984499990940094
Epoch 2, Loss: 0.00, Accuracy: 0.9984666705131531, Test Loss: 0.09109530597925186, Test Accuracy: 0.984000027179718
Epoch 3, Loss: 0.00, Accuracy: 0.9987333416938782, Test Loss: 0.08496914058923721, Test Accuracy: 0.9853000044822693
Epoch 4, Loss: 0.00, Accuracy: 0.9986333250999451, Test Loss: 0.09892162680625916, Test Accuracy: 0.9818000197410583
Epoch 5, Loss: 0.01, Accuracy: 0.998283326625824, Test Loss: 0.10027451813220978, Test Accuracy: 0.9835000038146973
Epoch 6, Loss: 0.00, Accuracy: 0.9991666674613953, Test Loss: 0.10565908253192902, Test Accuracy: 0.9825999736785889
